# Antigravity Schema Registry

**Purpose**: Introspect Snowflake schema to build the foundation for AI-powered SQL generation.

**Critical Rule**: Antigravity REFUSES to generate SQL without validating schema first.

This notebook:
1. Fetches schema from `DATA_QUALITY_DB.INFORMATION_SCHEMA.COLUMNS`
2. Builds a comprehensive registry of tables and columns
3. Validates registry completeness
4. Exports schema for LLM consumption

In [ ]:
# Import required libraries
import snowflake.connector
import pandas as pd
import json
from collections import defaultdict
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")

In [ ]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
    database='DATA_QUALITY_DB',
    schema='DB_METRICS'
)

cursor = conn.cursor()
print("✓ Connected to Snowflake")

In [ ]:
# Fetch schema from INFORMATION_SCHEMA
schema_sql = """
SELECT
  TABLE_CATALOG,
  TABLE_SCHEMA,
  TABLE_NAME,
  COLUMN_NAME,
  DATA_TYPE,
  IS_NULLABLE,
  ORDINAL_POSITION
FROM DATA_QUALITY_DB.INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_SCHEMA = 'DB_METRICS'
ORDER BY TABLE_SCHEMA, TABLE_NAME, ORDINAL_POSITION
"""

cursor.execute(schema_sql)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=columns)
print(f"✓ Fetched {len(df)} column definitions")
df.head(10)

In [ ]:
# Build schema registry
schema_registry = defaultdict(lambda: defaultdict(list))

for _, row in df.iterrows():
    db_name = row['TABLE_CATALOG']
    schema_name = row['TABLE_SCHEMA']
    table_name = row['TABLE_NAME']
    column_name = row['COLUMN_NAME']
    data_type = row['DATA_TYPE']
    is_nullable = row['IS_NULLABLE']
    ordinal_position = row['ORDINAL_POSITION']
    
    # Use fully qualified schema name as key
    fq_schema = f"{db_name}.{schema_name}".upper()
    
    schema_registry[fq_schema][table_name].append({
        'columnName': column_name,
        'dataType': data_type,
        'isNullable': is_nullable == 'YES',
        'ordinalPosition': ordinal_position
    })

print(f"✓ Built schema registry with {len(schema_registry)} schemas")
print(f"  Schemas: {list(schema_registry.keys())}")

In [ ]:
# Validate registry completeness
required_tables = [
    'DQ_METRICS',
    'DQ_AI_INSIGHTS',
    'DQ_LLM_EXECUTION_LOG'
]

missing_tables = []
for table in required_tables:
    found = False
    for schema_name, tables in schema_registry.items():
        if table in tables:
            found = True
            break
    if not found:
        missing_tables.append(table)

if missing_tables:
    print(f"⚠ WARNING: Missing required tables: {missing_tables}")
    print("  Run sql/setup_antigravity_observability.sql to create them")
else:
    print("✓ All required tables present")

In [ ]:
# Display table summary
print("\n=== Schema Registry Summary ===")
for schema_name, tables in schema_registry.items():
    print(f"\n{schema_name}:")
    for table_name, columns in tables.items():
        print(f"  {table_name} ({len(columns)} columns)")
        for col in columns[:5]:  # Show first 5 columns
            nullable = "NULL" if col['isNullable'] else "NOT NULL"
            print(f"    - {col['columnName']} ({col['dataType']}) {nullable}")
        if len(columns) > 5:
            print(f"    ... and {len(columns) - 5} more columns")

In [ ]:
# Export schema for LLM consumption (plain text format)
llm_schema_text = []
for schema_name, tables in schema_registry.items():
    for table_name, columns in tables.items():
        col_list = ', '.join([f"{c['columnName']} ({c['dataType']})" for c in columns])
        llm_schema_text.append(f"{schema_name}.{table_name}: {col_list}")

schema_text = '\n'.join(llm_schema_text)

# Save to file
with open('../prompts/schema_registry.txt', 'w') as f:
    f.write(schema_text)

print("✓ Schema exported to ../prompts/schema_registry.txt")
print(f"  Total size: {len(schema_text)} characters")
print("\nPreview:")
print(schema_text[:500] + "...")

In [ ]:
# Export as JSON for programmatic use
with open('../prompts/schema_registry.json', 'w') as f:
    json.dump(dict(schema_registry), f, indent=2, default=str)

print("✓ Schema exported to ../prompts/schema_registry.json")

In [ ]:
# Cleanup
cursor.close()
conn.close()
print("✓ Connection closed")

## Next Steps

1. Review the schema registry output above
2. Ensure all required tables are present
3. Proceed to `02_llm_plan_generation.ipynb` to test plan generation

**Remember**: This schema registry is the anti-hallucination gate. Without it, Antigravity REFUSES to execute.